<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/Standard_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./2_1_OLCI_advanced_data_access_eumdac.ipynb" target="_blank"><< Advanced OLCI data access with the EUMDAC client</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./2_3_OLCI_SNAP_batch_processing_C2RCC.ipynb" target="_blank">Using SNAP to batch process OLCI with C2RCC >></a>

<font color="#138D75">**Copernicus Marine Training Service**</font> <br>
**Copyright:** 2023 EUMETSAT <br>
**License:** MIT <br>
**Authors:** Ben Loveday (EUMETSAT/Innoflair UG), Hayley Evers-King (EUMETSAT)

<html>
  <div style="width:100%">
    <div style="float:left"><a href="https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.eumetsat.int%2Feumetlab%2Foceans%2Focean-training%2Fsensors%2Flearn-olci/HEAD?urlpath=%2Ftree%2F2_OLCI_advanced%2F2_2_acquire_OLCI_time_series.ipynb"><img src="https://mybinder.org/badge_logo.svg" alt="Open in Binder"></a></div>
    <div style="float:left"><p>&emsp;</p></div>
  </div>
</html>

<div class="alert alert-block alert-success">
<h3>Learn OLCI: Advanced</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
Users should review the following notebooks for more information on setting up a credentials files for using the eumdac python library to retrieve OLCI data from the Data Store:
    
- **<a href="../1_OLCI_introductory/1_1a_OLCI_data_access_Data_Store.ipynb">1_1a_OLCI_data_access_Data_Store.ipynb</a>**

You will also need to have registered for a **<a href="https://eoportal.eumetsat.int/" target="_blank">A EUMETSAT Earth Observation Portal account</a>** to download from the EUMETSAT Data Store.

</div>
<hr>

# 2.2 Acquiring a regional OLCI times series

### Data used

| Product Description | Data Store collection ID| Product Navigator |
|:--------------------:|:-----------------------:|:-------------:|
| Sentinel-3 OLCI level-1B full resolution | EO:EUM:DAT:0409 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:OL_1_EFR___NTC?query=OLCI&filter=satellite__Sentinel-3&filter=instrument__OLCI&filter=processingLevel__Level%201%20Data&s=advanced" target="_blank">link</a> |
| Sentinel-3 OLCI level-2 full resolution | EO:EUM:DAT:0407 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:OL_2_WFR___NTC?query=OLCI&filter=satellite__Sentinel-3&filter=instrument__OLCI&filter=processingLevel__Level%202%20Data&s=advanced" target="_blank">link</a> |

### Learning outcomes

At the end of this notebook you will know;
* <font color="#138D75">**Search**</font> for a time series of OLCI data (L1B or L2) for your region of interest using the EUMETSAT Data Store API client (`eumdac`)
* <font color="#138D75">**Download**</font> this time series

### Outline

The EUMETSAT Data Store offers many ways to interact with data in order to refine searches. Many of these methods are supported by the EUMETSAT Data Access Client (`eumdac`). In this notebook we will showcase some of the possibilities for using `eumdac` to better interact with OLCI collections. This notebook assumes that you already have an understanding of the available Data Store interfaces, which you can gain by running the **<a href="../1_OLCI_introductory/1_1a_OLCI_data_access_Data_Store.ipynb">1_1a_OLCI_data_access_Data_Store</a>** notebook.


<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
1. [Step 1: Authenticating the API](#section1)
1. [Step 2: Creating a search by collection](#section2)
1. [Step 3: Filtering by time](#section3)
1. [Step 4: Filtering by space](#section4)
1. [Step 5: Filtering by timeliness](#section5)
1. [Step 6: Filtering by satellite](#section6)
1. [Step 7: Removing NTC duplicates](#section7)
1. [Step 8: Downloading, filtering by polygon overlap](#section8)

<hr>

When we use Python, we nearly always start by importing the libraries we need. Each library gives us access to the capabilities we need to perform specific tasks. We can import library as shown in the box below.

In [2]:
import datetime     # a libary that allows us to work with dates and times
import json         # a library that helps us make JSON format files
import numpy as np  # a library that lets us work with arrays; we import this with a new name "np"
import os           # a library that allows us access to basic operating system commands like making directories
from shapely import geometry # a library that support construction of geometry objects
import shutil       # a library that allows us access to basic operating system commands like copy
import xml.etree.ElementTree as ET # a library that allows us to work with XML files
import zipfile      # a library that allows us to unzip zip-files.
import eumdac       # a tool that helps us download via the eumetsat/data-store

Next we will create a download directory to store the products we will download in this notebook.

In [15]:
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## <a id='section1'></a>Step 1: Authenticating the API
[Back to top](#TOC_TOP)

</div>

Before we use the Data Store to download data, we must first authenticate our access and retrieve an access token. More **essential** information on setting this up can be found in the **<a href="../1_OLCI_introductory/1_1a_OLCI_data_access_Data_Store.ipynb">1_1a_OLCI_data_access_Data_Store</a>** notebook.

In [3]:
# load credentials
with open(os.path.join(os.path.expanduser("~"),'.eumdac_credentials')) as json_file:
    credentials = json.load(json_file)
    token = eumdac.AccessToken((credentials['consumer_key'], credentials['consumer_secret']))
    print(f"This token '{token}' expires {token.expiration}")

# create data store object
datastore = eumdac.DataStore(token)

This token '9eb7d22d-311f-3239-927b-2766c192f2e6' expires 2022-07-18 09:49:13.412403


<div class="alert alert-info" role="alert">

## <a id='section2'></a>Step 2: Creating a search by collection
[Back to top](#TOC_TOP)

</div>

We will work with the OLCI Level-1 full resolution data throughout this notebook. The collection ID for this data type is `EO:EUM:DAT:0409`. You can find this information on the Data Store (https://data.eumetsat.int/), or ask the `eumdac` client to tell you all the avaiable collections by calling the `eumdac.DataStore(token).collections` method.

First, we need to define our **collection ID**;

In [4]:
# set collection ID for OLCI L1 EFR
collectionID = 'EO:EUM:DAT:0409'

To filter by collection, we simply provide the collectionID to the `datastore.get_collection method`

In [5]:
# Use collection ID
selected_collection = datastore.get_collection(collectionID)
print(f"{selected_collection.title}\n---\n{selected_collection.abstract}")

OLCI Level 1B Full Resolution - Sentinel-3
---
OLCI (Ocean and Land Colour Instrument) Full resolution: 300m at nadir. Level 1 products are calibrated Top Of Atmosphere radiance values at OLCI 21 spectral bands. Radiances are computed from the instrument digital counts by applying geo-referencing, radiometric processing (non-linearity correction, smear correction, dark offset correction, absolute gain calibration adjusted for gain evolution with time), and stray-light correction for straylight effects in OLCI camera's spectrometer and ground imager. Additionally, spatial resampling of OLCI pixels to the 'ideal' instrument grid, initial pixel classification, and annotation at tie points with auxiliary meteorological data and acquisition geometry are provided. The radiance products are accompanied by error estimate products, however the error values are currently not available. - All Sentinel-3 NRT products are available at pick-up point in less than 3h. - All Sentinel-3 Non Time Critica

<div class="alert alert-info" role="alert">

## <a id='section3'></a>Step 3: Filtering by time
[Back to top](#TOC_TOP)

</div>

To filter by time, we can pass python *datetime* arguments to the **dtstart** and **dtend** arguments of our collection when using the `.search()` method. We construct these as shown below.

In [6]:
# time filter the collection for products
dtstart = datetime.datetime(2022, 6, 24, 0, 0)
dtend = datetime.datetime(2022, 6, 28, 23, 59)

Now lets apply these date/time bounds to our search.

In [7]:
products = selected_collection.search(dtstart=dtstart, dtend=dtend)
print(f"Found {len(products)} products")

Found 4590 products


*Note: can can see a full range of the search options available in a collection object by using the `selected_collection.search_options` method*

<div class="alert alert-info" role="alert">

## <a id='section4'></a>Step 4: Filtering by space
[Back to top](#TOC_TOP)

</div>

We can also add geographical filtering by passing in a <a href="https://en.wikipedia.org/wiki/Well-known_text_representation_of_geometry" target="_blank">Well Known Text</a> (WKT) format region of interest into the **geo** argument of the `.search()` method. The box below defines our WKT using the `shapely` geometry library, which is useful as it gives us a polygon object that we later use to compare our ROI area with the granules we are looking at.

In [8]:
# space/time filter the collection for products
north = 58.50
south = 53.75
east = 21.80
west = 17.50
ROI = [[west, south], [east, south], [east, north], [west, north], [west, south]]
ROI_WKT = geometry.Polygon([[p[0], p[1]] for p in ROI])

Now lets use this polygon to refine our search further...

In [9]:
products = selected_collection.search(
    geo=ROI_WKT,
    dtstart=dtstart, 
    dtend=dtend)
print(f"Found {len(products)} products")

Found 25 products


<div class="alert alert-info" role="alert">

## <a id='section5'></a>Step 5: Filtering by timeliness
[Back to top](#TOC_TOP)

</div>

In this instance, we are only interested in Non Time-Critical (NTC) OLCI products. As before, we can add this filter to our search as follows.

In [10]:
products = selected_collection.search(
    geo=ROI_WKT,
    dtstart=dtstart, 
    dtend=dtend, 
    timeliness='NT')
print(f"Found {len(products)} products")

Found 13 products


<div class="alert alert-info" role="alert">

## <a id='section6'></a>Step 6: Filtering by satellite
[Back to top](#TOC_TOP)

</div>

Similarly, in this instance, we are only interested in products from Sentinel-3A, so we can prune our list further using the `sat` option in our filter.

In [11]:
products = selected_collection.search(
    geo=ROI_WKT,
    dtstart=dtstart, 
    dtend=dtend, 
    timeliness='NT',
    sat="Sentinel-3A")
print(f"Found {len(products)} products")

Found 6 products


<div class="alert alert-info" role="alert">

## <a id='section7'></a>Step 7: Removing NTC duplicates
[Back to top](#TOC_TOP)

</div>

Sometimes, during NTC processing, we have to revisit some granules to process them further. This is typically due to instances where we have to wait for more ancillary data. In this case, two NTC products are available on the Data Store. We only want the final product, which always has a later time. The box below will filter for the latest file only, taking advantage of the Data Stores default to show most recent products first.

In [12]:
processed_list = []
final_products = []
for product in products:
    file_tags = str(product).split('_')
    file_tags = [i for i in file_tags if i]
    granule_start = file_tags[4]
    if granule_start not in processed_list:
        final_products.append(product)
        processed_list.append(granule_start)
        
print(f"Found {len(final_products)} products")

Found 5 products


<div class="alert alert-info" role="alert">

## <a id='section8'></a>Step 8: Downloading, filtering by polygon overlap
[Back to top](#TOC_TOP)

</div>

We have now narrowed our search to a small number of polygons, but some of them may only slightly intersect with our region of interest. We can filter by percentage overlap by pre-checking the manifest file, and discarding the granules that don't meet a required overlap threshold. We will set this at 25%.

In [13]:
overlap_threshold = 25

Now we have defined our overlap we can download the data. The box below will do this, but requires some explanation.

* Firstly, in the first line, we set up a loop to iterate over each of our products of interest.
* Secondly, we read **only** the XML manifest file from each product. This manifest contains the footprint of the product, which we can compare with the ROI we defined above, calculating the percentage overlap between the two **pc_overlap**
* Thirdly, if the overlap percentage is greater than our threshold, we download the product by reading it into our download directory. The product arrives as a zip file, so we also unzip it and then clean up.

In [14]:
# 1. set up the product loop
for final_product in final_products:
    
    # 2. this reads the XML into memory and finds the correct part for the polygon and compares against our reference
    for entry in final_product.entries:
        if 'xfdumanifest.xml' in entry:
            with final_product.open(entry=entry) as fsrc:
                tree = ET.ElementTree(ET.fromstring(fsrc.data))
                root = tree.getroot()
                polygon = np.asarray(root.findall('.//gml:posList',
                    {'gml':"http://www.opengis.net/gml"})[0].text.split(' ')).astype(float)
                this_polygon = geometry.Polygon(list(zip(polygon[1::2], polygon[0::2])))
                pc_overlap = ROI_WKT.intersection(this_polygon).area/ROI_WKT.area*100

    # 3. if our overlap is greater than the threshold, we get the download the data and unzip it
    if pc_overlap > overlap_threshold:
        print(f"Percentage overlap: {int(pc_overlap)}%, downloading product")

        # download the zip file
        with final_product.open() as fsrc, open(os.path.join(download_dir, fsrc.name), mode='wb') as fdst:
            print(f'Downloading {fsrc.name}.')
            shutil.copyfileobj(fsrc, fdst)
            print(f'Download of product {fsrc.name} finished.')

        # unzip the file
        with zipfile.ZipFile(fdst.name, 'r') as zip_ref:
            for file in zip_ref.namelist():
                if file.startswith(str(final_product)):
                    zip_ref.extract(file, download_dir)
            print(f'Unzipping of product {fdst.name} finished.')

        # clean up
        os.remove(fdst.name)

Percentage overlap: 100%, downloading product
Download of product S3A_OL_1_EFR____20220628T085911_20220628T090211_20220629T160353_0180_087_050_1980_MAR_O_NT_002.SEN3.zip finished.
Unzipping of product /Users/benloveday/Code/Git_Reps/CMTS/internal/sensors/learn-olci/2_OLCI_advanced/products/S3A_OL_1_EFR____20220628T085911_20220628T090211_20220629T160353_0180_087_050_1980_MAR_O_NT_002.SEN3.zip finished.
Percentage overlap: 100%, downloading product
Download of product S3A_OL_1_EFR____20220627T092522_20220627T092822_20220630T140909_0179_087_036_1980_MAR_O_NT_002.SEN3.zip finished.
Unzipping of product /Users/benloveday/Code/Git_Reps/CMTS/internal/sensors/learn-olci/2_OLCI_advanced/products/S3A_OL_1_EFR____20220627T092522_20220627T092822_20220630T140909_0179_087_036_1980_MAR_O_NT_002.SEN3.zip finished.
Percentage overlap: 71%, downloading product
Download of product S3A_OL_1_EFR____20220626T095133_20220626T095433_20220627T152906_0179_087_022_1980_MAR_O_NT_002.SEN3.zip finished.
Unzipping o

If all went well, your products are now downloaded. You can find them in the `products` directory in the same folder as this notebook.

<div class="alert alert-block alert-warning">

### Challenge:

Now you have run this for acquiring the OLCI L1 full resolution (EFR) data, can you do adapt the script above to get the corresponding granules for the OLCI L2 full resolution (WFR) granules?

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./2_1_OLCI_advanced_data_access_eumdac.ipynb" target="_blank"><< Advanced OLCI data access with the EUMDAC client</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./2_3_OLCI_SNAP_batch_processing_C2RCC.ipynb" target="_blank">Using SNAP to batch process OLCI with C2RCC >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean" target="_blank">View on GitLab</a> | <a href="https://training.eumetsat.int/" target="_blank">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int target="_blank">Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int target="_blank">Contact our training team to collaborate on and reuse this material</a></span></p>